In [15]:
import re
import pandas as pd

# Processamento dos textos
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import RSLPStemmer
import spacy

## Baixar recursos da NLTK
nltk.download('punkt')
nltk.download('rslp')

# Similaridade
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

## Carregar o modelo de língua portuguesa do spaCy
nlp = spacy.load('pt_core_news_sm')


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/kandarpagalas/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package rslp to
[nltk_data]     /Users/kandarpagalas/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


In [19]:
# Lê o dataset de vagas
dataset_df = pd.read_csv("data/eng_de_dados.csv")

# Renomeia coluna e seta índice
dataset_df = dataset_df.rename(columns={"Unnamed: 0": "merging_id"})
dataset_df = dataset_df.set_index("merging_id")

dataset_df.sample()


,titulo,empresa_nome,localidade,tempo_em_aberto,empresa_link,workplace_type,senioridade,jobs_description,skills,id,url,qtd_candidatos,recrutador
merging_id,,,,,,,,,,,,,
136,CONSULTOR DE DATA SCIENCE SAO PAULO,Management Solutions,São Paulo e Região,Anunciada novamente há 2 semanas,https://www.linkedin.com/company/management-so...,Híbrido,Assistente,Management Solutions é uma firma internacional...,Aprendizado de máquina;Big data;Ciência de dad...,3854295784,https://www.linkedin.com/jobs/view/3854295784,Mais de 100 candidaturas,NaN


# Similaridade entre sessão SOBRE do linkedin e descrição das vagas usando TF-IDF
Utilizarei o TF-IDF para buscar similaridade entre a sessão SOBRE de um perfil no linkedin com a descrição das vagas. Em seguida ranquear as 10 mais similares e mostrar o resultado

#### Cálculo TF-IDF

In [29]:
def calcular_tf_idf_e_similaridade(frases, frase_teste):
    vectorizer = TfidfVectorizer()

    tfidf_matrix_treino = vectorizer.fit_transform(frases)
    tfidf_array_treino = tfidf_matrix_treino.toarray()

    tfidf_matrix_teste = vectorizer.transform(frase_teste)
    tfidf_array_teste = tfidf_matrix_teste.toarray()

    similaridade = cosine_similarity(tfidf_array_teste, tfidf_array_treino)

    # return pd.DataFrame(similaridade, columns=frases)

    return pd.DataFrame(similaridade, columns=frases, index=["similaridade"])
    

In [32]:
# Sesão SOBRE do linkedin
candidate_about = [
    """
    Profissional especializado em Engenharia de Dados, com base acadêmica multidisciplinar, tendo formação em Arquitetura e Urbanismo, Ciências da Computação e Biologia. Possui +2 anos de experiência prática em desenvolvimento Web, Pipelines de Dados, ETL/ELT, WebScraping e automação de processos. Iniciou a transição de carreira para a Engenharia de Dados ao identificar sua paixão pelos desafios da programação com foco em dados. Atualmente buscando aperfeiçoamento com o MBA em Engenharia de Dados

    # HABILIDADES CHAVE
    Pipeline de dados | WebScraping | Automação
    Data Warehouse | Data Lake | ETL | ELT
        
    # LINGUAGENS DE PROGRAMAÇÃO
    Python | SQL | Javascript | HTML | CSS
    """
]

# Exporta as descrições das vagas para uma list
descriptions = dataset_df["jobs_description"].to_list()

# Calcula similaridade
resultado_similaridade = calcular_tf_idf_e_similaridade(descriptions, candidate_about)

# Amostra do resultado em formato colunar
display(resultado_similaridade.T.reset_index().sample(5))

,index,similaridade
90,Que tal se juntar ao nosso time? Venha fazer p...,0.333738
8,Faça parte do time de Heróis de Sangue Amarelo...,0.339323
143,Somos especialistas digitais e impactamos as m...,0.287383
70,"Job Description Somos mais que uma máquina, so...",0.261618
159,Are you a tech professional looking for remote...,0.012688


#### Ranqueamente e recuperação das vagas com maior similaridade

In [37]:
# Transpondo a matrix para que as similaridades fiquem em formato colunar
# Reset do index para poder fazer o merge
similaridade_to_merge = resultado_similaridade.T.reset_index()

# Drop da coluna de texto, não será necessária após o merge
similaridade_to_merge = similaridade_to_merge.drop(columns=["index"], axis=1)

# Merge dos df
merged = pd.merge(dataset_df, similaridade_to_merge, left_index=True, right_index=True)

# Ordena pela similaridade de forma decrescente
merged = merged.sort_values(by='similaridade', ascending=False)

# Faz select para mostrar apenas colunas de interesse
filtered_df = merged[["similaridade","titulo", "senioridade", "workplace_type", "url"]]

# Aplica filtro para vagas workplace_type = remoto
# filtered_df = filtered_df[filtered_df["workplace_type"] == "Remoto"]

display(filtered_df.head(10))

,similaridade,titulo,senioridade,workplace_type,url
merging_id,,,,,
252,0.409036,Analista de Dados e BI (Data Lake),Pleno-sênior,Híbrido,https://www.linkedin.com/jobs/view/3839546618
45,0.371854,Desenvolvedor em Engenharia de Dados,Pleno-sênior,Tempo integral,https://www.linkedin.com/jobs/view/3907132071
115,0.367727,Data Scientist Specialist,Assistente,Tempo integral,https://www.linkedin.com/jobs/view/3838826786
336,0.367407,Arquiteto de Dados - Tech Lead,Pleno-sênior,Presencial,https://www.linkedin.com/jobs/view/3880479062
355,0.359501,Data Engineer,Pleno-sênior,Remoto,https://www.linkedin.com/jobs/view/3879625117
231,0.359169,Pessoa Engenheira de Dados Sênior,Pleno-sênior,Híbrido,https://www.linkedin.com/jobs/view/3900067274
206,0.359169,Pessoa Engenheira de Dados Sênior,Pleno-sênior,Híbrido,https://www.linkedin.com/jobs/view/3900067274
25,0.356134,Analista de dados,Pleno-sênior,Presencial,https://www.linkedin.com/jobs/view/3902444095
23,0.356134,Analista de dados,Pleno-sênior,Presencial,https://www.linkedin.com/jobs/view/3902444095


# Analise das skills mais predominantes
Implementei um contador de palavras para contar e ordenar as skills

In [38]:
# Exporta as descrições das vagas para uma list
skills_df = dataset_df[["senioridade", "skills"]]

# Remove linhas com skills vazias
skills_df = skills_df.dropna()

# Aplica filtro por senioridade
senioridade = "pleno"
pattern = re.compile(rf'\b{senioridade}\b', re.IGNORECASE)
skills_df = skills_df[skills_df["senioridade"].str.contains(pattern)]

# Exporta as skills das vagas para uma list
skills_list = skills_df["skills"].to_list()

# Prepara a string para ser quebrada
def clean_skill_list(x):
    try:
        clean = x.replace("[","")
        clean = clean.replace("]","")
        clean = clean.replace("'","")
        clean_list = list(map(lambda x: x.strip(), clean.split(";")))
        return clean_list
    except:
        print(x)
        return []


# Une todas as skills em uma lista
all_skills = []
for row in skills_list:
    all_skills.extend(clean_skill_list(row))

# Conta a ocorrência de uma skill
skills_counter = {}
for skill in all_skills:
    if skill in skills_counter:
        skills_counter[skill] += 1
    else:
        skills_counter[skill] = 1

# Cria um dataframe
skills_count_df = pd.DataFrame(
    list(skills_counter.items()),
    columns = ["skill", "count"]
)

# Ordena pelo número de ocorrências
skills_count_df = skills_count_df.sort_values(by='count', ascending=False)

# Mostra o top 20
display(skills_count_df.head(50))

,skill,count
6,Ciência de dados,90
72,Engenharia de dados,82
5,Banco de dados,76
31,Python,61
3,Analítica de dados,54
15,SQL,54
28,Análise de dados,49
7,Modelagem de dados,41
145,Comunicação,38
12,"ETL (Extração, transformação e carregamento)",34
